<h1> TDDE16 Project <h1>

<h2>Goal of this project:</h2>

<br> (1) Access the data from the dataset
<br> (2) Sort by genre and age of the song
<br> (3) Analyze it with LDA (1 way)
<br> (4) Analyze it with BERT (2 way)
<br> (5) Compare it to prediction/forecast/ thesis
<br> (6) Analyse
<br> (7) How to deal with things like beleidigungen

<h2> Prediction/ Forecast/ Thesis / Hypotheses: </h2>

<br> 1. Music is based around politcal topics and things people car the moment (Buzz words from Google in that era)
<br> 2. Love songs are always important
<br> 3. Love songs do not differ to much (how to analyze/proof)
<br> 4. Songs have less information in them
<br> 5. Newer Songs are more repetitive than older ones (Reddit inspiration), https://www.nature.com/articles/s41598-024-55742-x
<br> 6. How did the chorus change over time?
<br> 7. Song did get easier to understand over time -> how to calculate?
<br> 8a. Topic Heat Map: Topics across a genre a different

----

<br> 8. Take one artist per genre and compare it songs over time example eminem from 2000 and now

<h3>(1) Access the data from the dataset </h3>


In [1]:
%%capture
%pip install datasets

from datasets import load_dataset

ds = load_dataset("sebastiandizon/genius-song-lyrics")

In [2]:
def filter_songs_by_year(example):
    # Sicherstellen, dass ein Datum vorhanden ist
    if 'year' in example and example['year']:
        # Extrahiere das Jahr (angenommen, release_date ist im Format 'YYYY-MM-DD')
        year = int(example['year'])
        if year >2000 and year <2010:
            return 2000 <= year <= 2010
    return False

# Songs filtern
filtered_songs = ds.filter(filter_songs_by_year)

# Anzahl der gefilterten Songs anzeigen
print(f"Gefundene Songs zwischen 2000-2010: {len(filtered_songs)}")

Gefundene Songs zwischen 2000-2010: 1


In [3]:
list_songs = []
for i in range(100):
    if ds['train'][i]['year'] > 2000:
        list_songs.append(ds['train'][i])

In [4]:
len(list_songs)

67

In [5]:
# Assuming the dataset has a 'category' column
unique_categories = ds['train'].unique('tag')  # Get all unique categories

# Group into subdatasets
subdatasets = {category: ds['train'].filter(lambda x: x['tag'] == category and x['language'] == 'en') for category in unique_categories}

# Example: Print the first entry of each subdataset
for category, subdataset in subdatasets.items():
    print(f"Category: {category}, Number of entries: {len(subdataset)}")

Category: rap, Number of entries: 964605
Category: rb, Number of entries: 155082
Category: rock, Number of entries: 633308
Category: pop, Number of entries: 1393559
Category: misc, Number of entries: 140986
Category: country, Number of entries: 86658


In [6]:
print(subdatasets.items())

dict_items([('rap', Dataset({
    features: ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'id', 'language_cld3', 'language_ft', 'language'],
    num_rows: 964605
})), ('rb', Dataset({
    features: ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'id', 'language_cld3', 'language_ft', 'language'],
    num_rows: 155082
})), ('rock', Dataset({
    features: ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'id', 'language_cld3', 'language_ft', 'language'],
    num_rows: 633308
})), ('pop', Dataset({
    features: ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'id', 'language_cld3', 'language_ft', 'language'],
    num_rows: 1393559
})), ('misc', Dataset({
    features: ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'id', 'language_cld3', 'language_ft', 'language'],
    num_rows: 140986
})), ('country', Dataset({
    features: ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'id', 'la

In [7]:
#rap_dataset =  subdatasets['rap'].sort('views', reverse=True)

In [8]:
tsd_rap = subdatasets['rap'][:2]
print(tsd_rap)

{'title': ['Killa Cam', 'Can I Live'], 'tag': ['rap', 'rap'], 'artist': ["Cam'ron", 'JAY-Z'], 'year': [2004, 1996], 'views': [173166, 468624], 'features': ['{"Cam\\\\\'ron","Opera Steve"}', '{}'], 'lyrics': ['[Chorus: Opera Steve & Cam\'ron]\nKilla Cam, Killa Cam, Cam\nKilla Cam, Killa Cam\nKilla Cam, Cam\nKilla Cam, Killa Cam, Cam\nKilla Killa Killa Cam\nKilla Cam, Cam, Killa (Killa!)\nKilla Cam, Killa Cam, Cam (Bases loaded)\nKilla Cam, Killa Cam (Uh-huh)\nKilla Cam, Cam (Santana on second, Jim on third)\nKilla Cam, Killa Cam, Cam (I\'m at bat)\nKilla Killa Killa Cam\nKilla Cam, Cam, Killa (I\'m \'bout to hit this shit out the world)\nKilla Cam (Ugh, Heatmakerz), Killa Cam, Cam\nKilla Cam, Killa Cam\nKilla Cam, Cam (Hahahaha)\nKilla Cam, Killa Cam, Cam\nKilla Killa Killa Cam\nKilla Cam, Cam, Killa (We  make this shit clap)\nKilla Cam, Killa Cam, Cam\nKilla Cam, Killa Cam\nKilla Cam, Cam\nKilla Cam, Killa Cam, Cam\nKilla Killa Killa Cam (Killa! Killa!)\nKilla Cam, Cam, Killa\n[Verse 1

In [9]:
# This code will use stopwords for lemmatization

import spacy
from nltk.corpus import stopwords

# Load the English NLP model from spacy
nlp = spacy.load("en_core_web_sm")

# Load English stop words from nltk
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    doc = nlp(text)
    
    # Lemmatize and remove stop words
    lemmatized_text = " ".join(
        token.lemma_ for token in doc
        if token.text.lower() not in stop_words and token.is_alpha
    )
    
    return lemmatized_text

In [10]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
#print(type(subdatasets['rap']))
#print(len(subdatasets['rap']['lyrics']))
#lyrics_list = [entry['lyrics'] for entry in subdatasets['rap'][:10]]
rap = subdatasets['pop']['lyrics'][:1000]

rap_lemma = rap
i = 0
for r in rap:
    rap_lemma[i] = preprocess_text(r)
    i = i +1

# Initialize your topic model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
topic_model = BERTopic(embedding_model=embedding_model)
# Fit and transform the text data
topics, probs = topic_model.fit_transform(rap_lemma)

# Print or analyze the results
print("Topics:", topics)
print("Probabilities:", probs)

#print(rap_lemma[1])
#print(rap[1])

/usr/local/lib/python3.10/dist-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


Topics: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

<h2>LDA </h2>

In [11]:
import gensim
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# Beispiel-Liste von Strings


# Stopwords definieren
stop_words = set(stopwords.words('english'))

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess(doc):
    tokens = word_tokenize(doc.lower())  # Kleinbuchstaben und Tokenisierung
    #tokens = [word for word in tokens if word.is_alpha()]  # Nur alphabetische Wörter
    tokens = [word for word in tokens if word not in stop_words]  # Stopwords entfernen
    return tokens

# Preprocessing für alle Dokumente anwenden
processed_docs = [preprocess(doc) for doc in rap_lemma]

# Erstellen eines Wörterbuchs
dictionary = Dictionary(processed_docs)

# Erstellen eines Bag-of-Words-Korpus
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# LDA-Modell trainieren
lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,  # Anzahl der Themen
    random_state=42,
    passes=40,  # Anzahl der Durchgänge
    alpha='auto'  # Dirichlet-Hyperparameter
)
id2word = dictionary
# Themen anzeigen
print("Themen und ihre Schlüsselwörter:")
for idx, topic in lda_model.print_topics(num_words=5):
    print(f"Topic {idx}: {topic}")

# Dokument-Zuordnung zu Themen
#print("\nDokument-Themen-Zuordnung:")
#for doc_id, doc in enumerate(corpus):
#    print(f"Dokument {doc_id}: {lda_model.get_document_topics(doc)}")


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Themen und ihre Schlüsselwörter:
Topic 0: 0.023*"go" + 0.020*"chorus" + 0.020*"know" + 0.019*"love" + 0.015*"verse"
Topic 1: 0.122*"oh" + 0.053*"na" + 0.019*"wan" + 0.016*"get" + 0.014*"come"
Topic 2: 0.022*"go" + 0.018*"eh" + 0.017*"like" + 0.016*"oh" + 0.016*"get"
Topic 3: 0.042*"yeah" + 0.041*"get" + 0.024*"love" + 0.016*"like" + 0.014*"chorus"
Topic 4: 0.030*"ooh" + 0.029*"like" + 0.026*"girl" + 0.020*"get" + 0.013*"need"


In [12]:
#%pip install pyLDAvis==3.3.1
#import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.gensim 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.078835  0.102325       1        1  40.020899
4     -0.007793 -0.104257       2        1  19.635159
1     -0.157287 -0.043876       3        1  16.248118
3     -0.058334  0.113325       4        1  15.183787
2      0.144579 -0.067517       5        1   8.912038, topic_info=     Term         Freq        Total Category  logprob  loglift
145    oh  5079.000000  5079.000000  Default  30.0000  30.0000
207    na  2374.000000  2374.000000  Default  29.0000  29.0000
375  yeah  2550.000000  2550.000000  Default  28.0000  28.0000
506   ooh  1536.000000  1536.000000  Default  27.0000  27.0000
190   get  4093.000000  4093.000000  Default  26.0000  26.0000
..    ...          ...          ...      ...      ...      ...
223  take   100.634564  1311.730914   Topic5  -5.2180  -0.1498
375  yeah   103.592317  2550.993841   Topic5  -5.1890  -0.7860
281  love    93.028578  3021.301053   Topic5  -5.2966  -1.0628
392   one    85.380549  1402.804834   Topic5  -5.3824  -0.3813
51   make    84.252647  1471.142869   Topic5  -5.3957  -0.4422

[424 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
9567      1  0.958715  adios
1031      1  0.056170     ah
1031      2  0.262126     ah
1031      3  0.591344     ah
1031      4  0.067092     ah
...     ...       ...    ...
237       4  0.162675     yo
237       5  0.067781     yo
5599      2  0.963560    yuh
5599      5  0.029801    yuh
9029      5  0.961863   zayn

[864 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 4, 3])

<h2> Remove the bad words</h2>

In [13]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("tushifire/ldnoobw")
#file = kagglehub.load_dataset("tushifire/ldnoobw",path)

print("Path to dataset files:", path)

swear_words_file = os.path.join(path, 'en')  # Replace 'en.txt' with the correct filename

with open(swear_words_file, 'r') as f:
    swear_words = set(f.read().splitlines())

print(f"Loaded {len(swear_words)} swear words.")

print(type(swear_words))



Path to dataset files: /home/maurice/.cache/kagglehub/datasets/tushifire/ldnoobw/versions/2
Loaded 403 swear words.
<class 'set'>


In [16]:
import gensim
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# Beispiel-Liste von Strings


# Stopwords definieren
stop_words = set(stopwords.words('english'))

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess(doc):
    tokens = word_tokenize(doc.lower())  # Kleinbuchstaben und Tokenisierung
    #tokens = [word for word in tokens if word.is_alpha()]  # Nur alphabetische Wörter
    tokens = [word for word in tokens if word not in stop_words]  # Stopwords entfernen
    return tokens

def sort_out_bad_words(doc,word_list, replacement="swear"):
    """
    Ersetzt Wörter in einem Text, wenn sie in einer Liste vorkommen.
    
    :param text: Der Eingabetext (String).
    :param word_list: Liste der Wörter, die ersetzt werden sollen.
    :param replacement: Der Text, durch den die Wörter ersetzt werden sollen.
    :return: Der bearbeitete Text.
    """
    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)

# Get rid of the bad words and replace it with "swear word"
without_swear = [sort_out_bad_words(doc, swear_words ) for doc in rap_lemma]

# Preprocessing für alle Dokumente anwenden
processed_docs = [preprocess(doc) for doc in rap_lemma]

# Erstellen eines Wörterbuchs
dictionary = Dictionary(processed_docs)

# Erstellen eines Bag-of-Words-Korpus
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# LDA-Modell trainieren
lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=10,  # Anzahl der Themen
    random_state=42,
    passes=40,  # Anzahl der Durchgänge
    alpha='auto'  # Dirichlet-Hyperparameter
)
id2word = dictionary
# Themen anzeigen
print("Themen und ihre Schlüsselwörter:")
for idx, topic in lda_model.print_topics(num_words=5):
    print(f"Topic {idx}: {topic}")

# Dokument-Zuordnung zu Themen
#print("\nDokument-Themen-Zuordnung:")
#for doc_id, doc in enumerate(corpus):
#    print(f"Dokument {doc_id}: {lda_model.get_document_topics(doc)}")


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Themen und ihre Schlüsselwörter:
Topic 0: 0.026*"go" + 0.025*"love" + 0.023*"chorus" + 0.022*"know" + 0.018*"verse"
Topic 1: 0.222*"oh" + 0.022*"na" + 0.018*"come" + 0.016*"yeah" + 0.014*"get"
Topic 2: 0.036*"dance" + 0.023*"beat" + 0.020*"like" + 0.019*"see" + 0.017*"know"
Topic 3: 0.078*"yeah" + 0.042*"get" + 0.034*"la" + 0.016*"chorus" + 0.015*"like"
Topic 4: 0.049*"ooh" + 0.041*"like" + 0.041*"girl" + 0.020*"get" + 0.020*"need"
Topic 5: 0.026*"want" + 0.022*"get" + 0.021*"eh" + 0.018*"ah" + 0.014*"time"
Topic 6: 0.041*"da" + 0.021*"bad" + 0.020*"like" + 0.014*"love" + 0.014*"weddi"
Topic 7: 0.015*"go" + 0.015*"chorus" + 0.013*"get" + 0.012*"verse" + 0.012*"know"
Topic 8: 0.099*"na" + 0.022*"get" + 0.021*"wan" + 0.016*"run" + 0.015*"ha"
Topic 9: 0.038*"get" + 0.038*"baby" + 0.025*"love" + 0.022*"go" + 0.018*"boy"


In [15]:
import pyLDAvis.gensim_models
import pyLDAvis.gensim 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.078835  0.102325       1        1  40.020899
4     -0.007793 -0.104257       2        1  19.635159
1     -0.157287 -0.043876       3        1  16.248118
3     -0.058334  0.113325       4        1  15.183787
2      0.144579 -0.067517       5        1   8.912038, topic_info=     Term         Freq        Total Category  logprob  loglift
145    oh  5079.000000  5079.000000  Default  30.0000  30.0000
207    na  2374.000000  2374.000000  Default  29.0000  29.0000
375  yeah  2550.000000  2550.000000  Default  28.0000  28.0000
506   ooh  1536.000000  1536.000000  Default  27.0000  27.0000
190   get  4093.000000  4093.000000  Default  26.0000  26.0000
..    ...          ...          ...      ...      ...      ...
223  take   100.634564  1311.730914   Topic5  -5.2180  -0.1498
375  yeah   103.592317  2550.993841   Topic5  -5.1890  -0.7860
281  love    93.028578  3021.301053   Topic5  -5.2966  -1.0628
392   one    85.380549  1402.804834   Topic5  -5.3824  -0.3813
51   make    84.252647  1471.142869   Topic5  -5.3957  -0.4422

[424 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
9567      1  0.958715  adios
1031      1  0.056170     ah
1031      2  0.262126     ah
1031      3  0.591344     ah
1031      4  0.067092     ah
...     ...       ...    ...
237       4  0.162675     yo
237       5  0.067781     yo
5599      2  0.963560    yuh
5599      5  0.029801    yuh
9029      5  0.961863   zayn

[864 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 4, 3])